* Download the latest release and unzip it. Make a new directory under the user directory an copy all the unzipped folders and files into it.
* Open the log file log4j.properties.template. Change INFO to WARN,  save and close the file. 
* In the same directory remove .template from every filename.
* Go to Control Panel -> System and Security -> System -> Advanced Settings -> Environment Variables and create a new user variable named SPARK_HOME. Fill the value-field with the directory to Spark, e.g. C:\Users\XXXXX\Spark.
* Add %SPARK_HOME%\bin to the path variable.
* Download winutils, extract the content to c:\winutils\bin
* Add HADOOP_HOME  -> C:\winutils and %HADOOP_HOME%\bin to the user/system variables
* Make sure you have installed java
* Open command line and run spark-shell. If that does not work: change into C:\Users\XXXX\Spark\bin and try again.

![image.png](Spark.JPG)

##### Leider kriege ich es auf dem notebook nicht mehr zum Laufen, deswegen habe ich in VSC weitergemacht... im Mai lief es hier noch.

import findspark  
findspark.init()  
from pyspark import SparkConf, SparkContext  
from operator import add  
import re  
from English_Stop_Words import english_stop_words  
  
conf = SparkConf()  
conf.set("spark.driver.host", "localhost")  
conf.setMaster('local')  
conf.setAppName('Spark Basics Exercise')  
sc = SparkContext(conf=conf)  
  
##### Einlesen des vorgesäuberten Textfiles ins RDD
rdd_text = sc.textFile("shakespeare_cleaned.txt")  
  
##### Textbereinigung und Zählen der verschiedenen Wörter:  
wordcount = rdd_text \  
    .flatMap(lambda line: line.split(' ')) \  
    .map(lambda word: word.lower()) \  
    .map(lambda word: re.sub(r'\'+[a-z]', '', word)) \  
    .map(lambda word: re.sub(r'[\n\b\f\r\t\v,.!?:;\"[\]{}()&\-\'\x00]', ' ', word)) \  
    .map(lambda word: word.replace('\"', '')) \  
    .map(lambda word: word.strip()) \  
    .map(lambda word: re.sub(r'\s+[a-z]+$', '', word)) \  
    .map(lambda word: (word, 1)) \  
    .reduceByKey(add)  
  
print("Anzahl der Wörter: " + str(wordcount.count()))  
  
##### Wir erstellen eine zweite Liste, bei der wir zum Vergleich die sog. stop-words und alle Wörter, die aus weniger als 3 Zeichen bestehen herausfiltern:  
filtered_wordcount = wordcount.filter(lambda x: len(x[0]) > 2 and not x[0] in english_stop_words)  
print("Anzahl der Wörter ohne Stop-Wörter: " + str(filtered_wordcount.count()))  

##### Wir erstellen 2 Listen:  
most_frequent_words_filtered = filtered_wordcount.takeOrdered(24, key=lambda x: -x[1])  
wordcount = wordcount.filter(lambda x: len(x[0]) > 0)  
most_frequent_words = wordcount.takeOrdered(24, key=lambda x: -x[1])  
  
##### Wir erstellen 2 Textfiles, einmal die ungefilterte und einmal die gefilterte Liste  
with open('most_frequent_words.txt', 'w') as filehandle:  
    line_number = 1  
    for count_item in most_frequent_words:  
        filehandle.write(str(line_number) + "\t" + '%s\t\t-\tOccurrences: %d\n' % (count_item[0], count_item[1]))  
        line_number = line_number +1  
  
with open('most_frequent_words_filtered.txt', 'w') as filehandle:  
    line_number = 1  
    for count_item in most_frequent_words_filtered:  
        filehandle.write(str(line_number) + "\t" + '%s\t\t-\tOccurrences: %d\n' % (count_item[0], count_item[1]))  
        line_number = line_number +1  

Output:  
Anzahl der Wörter: 24328  
Anzahl der Wörter ohne Stop-Wörter und mit mindestens 3 Zeichen: 23815  

Ungefiltert:  
  
1	the		-	Occurrences: 27319  
2	and		-	Occurrences: 26023  
3	i		-	Occurrences: 20691  
4	to		-	Occurrences: 19736  
5	of		-	Occurrences: 17471  
6	a		-	Occurrences: 14468  
7	you		-	Occurrences: 13631  
8	my		-	Occurrences: 12464  
9	that		-	Occurrences: 11281  
10	in		-	Occurrences: 11069  
11	is		-	Occurrences: 10722  
12	not		-	Occurrences: 8499  
13	with		-	Occurrences: 7779  
14	me		-	Occurrences: 7777  
15	it		-	Occurrences: 7711  
16	for		-	Occurrences: 7624  
17	be		-	Occurrences: 6870  
18	his		-	Occurrences: 6869  
19	your		-	Occurrences: 6645  
20	he		-	Occurrences: 6600  
21	this		-	Occurrences: 6589  
22	but		-	Occurrences: 6256  
23	have		-	Occurrences: 5888  
24	as		-	Occurrences: 5735  

Gefiltert  
  
1	thou		-	Occurrences: 5485  
2	thy		-	Occurrences: 4031  
3	shall		-	Occurrences: 3591  
4	thee		-	Occurrences: 3181  
5	lord		-	Occurrences: 3089  
6	king		-	Occurrences: 3038  
7	good		-	Occurrences: 2812  
8	sir		-	Occurrences: 2761  
9	come		-	Occurrences: 2508  
10	let		-	Occurrences: 2350  
11	love		-	Occurrences: 2173  
12	enter		-	Occurrences: 2106  
13	man		-	Occurrences: 2002  
14	hath		-	Occurrences: 1939  
15	like		-	Occurrences: 1702  
16	say		-	Occurrences: 1687  
17	know		-	Occurrences: 1667  
18	make		-	Occurrences: 1636  
19	did		-	Occurrences: 1626  
20	speak		-	Occurrences: 1169  
21	duke		-	Occurrences: 1127  
22	time		-	Occurrences: 1083  
23	father		-	Occurrences: 1067  
24	tell		-	Occurrences: 1064  

